In [1]:
data_config = {
    "data_name": "chengdu",
    "data_path": "/data/hetianran/didi/chengdu/gps_20161101",
    "data_size": 100000,
    "data_form": "roadnet",
    "vocab_size": 5000,  # set by data factory
}

prediction_task_config = {
    "task_name": "prediction",
    "train_mode": "pre-train",  # pre-train, fine-tune, test-only
    "dataset_prop": (0.8, 0.1, 0.1),
    "input_len": 10,
    "output_len": 1,  # only 1
}

similarity_task_config = {
    "task_name": "similarity",
    "train_mode": "test-only",
    "dataset_prop": (0, 0, 1),
    "variant": "cropped",  # cropped, distorted
    "sub-task": "kNN",  # kNN, CDD
}

filling_task_config = {
    "task_name": "filling",
    "train_mode": "pre-train",
    "dataset_prop": (0.9, 0.1, 0),
    "sub-task": "autoregressive",  # mlm, autoregressive
}

embedding_config = {
    "emb_name": "normal",
    "emb_dim": 256,
    "pre-trained": False,
    "embs_path": "",
}

encoder_config = {
    "encoder_name": "transformer",
    "num_layers": 6,
    "d_model": embedding_config["emb_dim"],
    "num_heads": 8,
    "d_ff": 2048,
    "dropout": 0.1,
}

trainer_config = {
    "model_path": "./resource/model/backbone/backbone.pth",
    "batch_size": 64,
    "learning_rate": 1e-4,
    "num_epochs": 10,
    "optimizer": "adam",
    "loss_function": "cross_entropy",
    "lr_scheduler": "step_lr",
}

config = {
    "data_config": data_config,
    "task_config": prediction_task_config,
    "embedding_config": embedding_config,
    "encoder_config": encoder_config,
    "trainer_config": trainer_config,
}

In [ ]:
from trajlib.model.model_factory import create_model
from trajlib.data.data_factory import create_data

data, _ = create_data(config)
tot = 0
for traj in data.original:
    tot += len(traj)
print(tot / len(data.original))

In [ ]:
from trajlib.model.embedding.embedding_trainer import EmbeddingTrainer
from trajlib.model.embedding.gnn import GAETrainer
from trajlib.model.embedding.node2vec import Node2VecTrainer
from trajlib.data.data_factory import create_data

mapper: dict[str, type[EmbeddingTrainer]] = {
    "node2vec": Node2VecTrainer,
    "gat": GAETrainer,
    "gcn": GAETrainer,
}

if embedding_config["pre-trained"]:
    _, graph_data = create_data(config)
    trainer = mapper[embedding_config["emb_name"]](embedding_config, graph_data)
    trainer.train()

In [2]:
from accelerate import notebook_launcher

from trajlib.runner.base_runner import BaseRunner


def accelerate_run(config):
    runner = BaseRunner(config)
    runner.run()

# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"
accelerate_run(config)

# notebook_launcher(accelerate_run, args=(config,), num_processes=4, use_port="29500")

# TODO debug embedding 预训练和 encoder 训练无法连续运行

Launching training on 4 GPUs.


Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/home/hetianran/.conda/envs/traj/lib/python3.10/site-packages/leuvenmapmatching/visualization.py:130: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(figwidth, height))
/home/hetianran/.conda/envs/traj/lib/python3.10/site-packages/leuvenmapmatching/visualization.py:130: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warni

error:no osmids!
error:no osmids!
error:no osmids!
error:no osmids!


W0530 14:43:41.464000 103172 site-packages/torch/multiprocessing/spawn.py:160] Terminating process 103485 via signal SIGTERM
W0530 14:43:41.469000 103172 site-packages/torch/multiprocessing/spawn.py:160] Terminating process 103486 via signal SIGTERM
W0530 14:43:41.471000 103172 site-packages/torch/multiprocessing/spawn.py:160] Terminating process 103487 via signal SIGTERM
E0530 14:43:41.926000 103172 site-packages/torch/distributed/elastic/multiprocessing/api.py:732] failed (exitcode: 1) local_rank: 0 (pid: 103484) of fn: accelerate_run (start_method: fork)
E0530 14:43:41.926000 103172 site-packages/torch/distributed/elastic/multiprocessing/api.py:732] Traceback (most recent call last):
E0530 14:43:41.926000 103172 site-packages/torch/distributed/elastic/multiprocessing/api.py:732]   File "/home/hetianran/.conda/envs/traj/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/api.py", line 687, in _poll
E0530 14:43:41.926000 103172 site-packages/torch/distributed/elasti

ChildFailedError: 
============================================================
accelerate_run FAILED
------------------------------------------------------------
Failures:
  <NO_OTHER_FAILURES>
------------------------------------------------------------
Root Cause (first observed failure):
[0]:
  time      : 2025-05-30_14:43:40
  host      : gpu19.buaanlsde.org
  rank      : 0 (local_rank: 0)
  exitcode  : 1 (pid: 103484)
  error_file: /tmp/torchelastic_fybjukk5/none_1rsjfktt/attempt_0/0/error.json
  traceback : Traceback (most recent call last):
    File "/home/hetianran/.conda/envs/traj/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 355, in wrapper
      return f(*args, **kwargs)
    File "/tmp/ipykernel_103172/480669042.py", line 7, in accelerate_run
      runner = BaseRunner(config)
    File "/home/hetianran/trajectory/TrajMM/trajlib/runner/base_runner.py", line 33, in __init__
      self.model = create_model(config)
    File "/ho